In [1]:
import torch

# Тесты моделей

In [2]:
rnn = torch.nn.LSTM(50, 20, 4)
input_tensor = torch.randn(5, 3, 50)
h0 = torch.randn(4, 3, 20)
c0 = torch.randn(4, 3, 20)
output, (hn, cn) = rnn(input_tensor, (h0, c0))
output.shape, hn.shape, cn.shape

(torch.Size([5, 3, 20]), torch.Size([4, 3, 20]), torch.Size([4, 3, 20]))

In [71]:
input_tensor = torch.randint(0, 10, (2, 5)).long()+4
padded = torch.nn.functional.pad(input=input_tensor, pad=(0, 5), mode='constant', value=2)
emb = torch.nn.Embedding(
    num_embeddings=14,
    embedding_dim=20,
    padding_idx=2
)
embeded = emb(input_tensor)
print(emb(input_tensor).shape)



torch.Size([2, 5, 20])


In [27]:
input_tensor

tensor([3, 8, 4, 1, 8, 8])

# Чистый Encoder

In [83]:
class EncoderLSTM(torch.nn.Module):
  def __init__(
    self, 
    embedding_size: int,  # размерность векторов в который преобразуется какждый элемент последовательности 
    hidden_size: int,  # размерность векторов на выходе
    num_layers: int,  # число векторов скрытых состояний
):
    super(EncoderLSTM, self).__init__()
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.LSTM = torch.nn.LSTM(
      self.embedding_size, 
      hidden_size, 
      num_layers
    )

  def forward(self, x: torch.tensor):
    outputs, (hidden_state, cell_state) = self.LSTM(x)

    return outputs, hidden_state, cell_state

encoder_embedding_size = 29   # Размер векторов--эмбелингов будет 29
hidden_size = 17  # размер эмбедингов на выходе будет 17 
num_layers = 7  # число скрытых состояний

encoder_lstm = EncoderLSTM(encoder_embedding_size, hidden_size, num_layers)

In [86]:
input_tensor = torch.randn(2, 10, 29)
outputs, hidden_state, cell_state = encoder_lstm(padded)
padded.shape, outputs.shape, hidden_state.shape, cell_state.shape 

(torch.Size([2, 10]),
 torch.Size([2, 10, 17]),
 torch.Size([7, 10, 17]),
 torch.Size([7, 10, 17]))

# Со встроенной векторизацией

In [84]:
class EncoderLSTM(torch.nn.Module):
  def __init__(
    self, 
    vocab_size: int,  # размер словаря
    embedding_size: int,  # размерность векторов в который преобразуется какждый элемент последовательности 
    hidden_size: int,  # размерность векторов на выходе
    num_layers: int,  # число скрытых состояний
    pad_idx: int=2,  # PAD-элемент
):
    super(EncoderLSTM, self).__init__()
    self.vocab_size = vocab_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.embedding = torch.nn.Embedding(
      self.vocab_size, 
      self.embedding_size, 
      padding_idx=pad_idx
    )
    self.LSTM = torch.nn.LSTM(
      self.embedding_size, 
      hidden_size, 
      num_layers
    )

  def forward(self, x: torch.tensor):
    embedding = self.embedding(x)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return outputs, hidden_state, cell_state

vocab_size_encoder = 23  # Пусть размер словаря будет 23
encoder_embedding_size = 29   # Размер векторов--эмбелингов будет 29
hidden_size = 17  # размер эмбедингов на выходе будет 17 
num_layers = 7  # число скрытых состояний

encoder_lstm = EncoderLSTM(vocab_size_encoder, encoder_embedding_size, hidden_size, num_layers)

In [85]:
input_tensor = torch.randint(4, 23, (2, 5)).long()  # каких-то 2-е последовательности длиной 5 с типом long. 
                                                    # Значения ограниченны с низу до 4 из предположения,что в (0, 4) - спец символы.
padded = torch.nn.functional.pad(
    input=input_tensor, 
    pad=(0, 5), 
    mode='constant', 
    value=2
)  # Падим последовательности с права на 0 символов-отступа с лева на 5.
   # В итоге имеем 2-е последовательности длиной 10
outputs, hidden_state, cell_state = encoder_lstm(padded)
padded.shape, outputs.shape, hidden_state.shape, cell_state.shape 

(torch.Size([2, 10]),
 torch.Size([2, 10, 17]),
 torch.Size([7, 10, 17]),
 torch.Size([7, 10, 17]))